## Luftdata med kartor

Simon Winter, Infontology http://infontology.org



I den här lektionen ska vi se hur man kan hitta data för luftföroreningar från det nätverk av sammankopplade sensorer som finns på Luftdata.se

In [1]:
import requests
import pandas as pd
import gmaps
from credentials import key

/Users/management/.local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# https://console.developers.google.com/apis/dashboard?project=natural-furnace-264420
gmaps.configure(api_key=key)

In [68]:
latitude = '55.611437'
longitude = '12.994264'
distance = '150' # Avstånd i kilometer

Med hjälp av koordinaterna så skapar vi urlen. Lämna tomma platser inom {} i textsträngen, och ge variabelnamnen i format-parentesen

In [69]:
url = 'http://api.luftdaten.info/v1/filter/area={},{},{}'.format(latitude, longitude, distance)
url ## Om man ger ett variabelnamn på sista raden av en cell skrivs värdet ut. Det går också att göra print(url)

'http://api.luftdaten.info/v1/filter/area=55.611437,12.994264,150'

Requests-biblioteket gör det lätt att hantera frågor till servern. 

In [70]:
response = requests.get(url)

In [71]:
sensors = response.json()
number_of_sensors = len(sensors)
number_of_sensors

146

Om number_of_sensors är större än 0 så finns det sensorer i listan. Då kan man få fram sensorvärdena på olika sätt. Det första värdet ligger alltid i sensors[0]

In [72]:
sens2 = pd.DataFrame(columns=['latitude', 'longitude'])

In [73]:
sens2

,latitude,longitude


In [74]:
def create_sensordict (sensor):
    sensordict = {}
    sensordict['longitude'] = sensor['location']['longitude']
    sensordict['latitude'] = sensor['location']['latitude']
    for stype in sensor['sensordatavalues']:
        sensordict[stype['value_type']] = stype['value']
    return sensordict

In [75]:
for sensor in sensors:
    sens2 = sens2.append (create_sensordict(sensor), ignore_index=True)

    

In [76]:
sens2

,latitude,longitude,humidity,temperature,P1,P2,pressure,pressure_at_sealevel
0,56.196,15.094,99.90,6.10,NaN,NaN,NaN,NaN
1,56.196,15.094,NaN,NaN,91.27,16.37,NaN,NaN
2,56.07,12.698,99.90,8.70,NaN,NaN,NaN,NaN
3,56.07,12.698,NaN,NaN,11.47,9.63,NaN,NaN
4,55.674,13.074,NaN,NaN,60.90,18.60,NaN,NaN
...,...,...,...,...,...,...,...,...
141,55.722,13.202,99.90,9.00,NaN,NaN,NaN,NaN
142,55.882,14.228,NaN,NaN,41.73,16.50,NaN,NaN
143,55.722,13.202,NaN,NaN,34.75,19.50,NaN,NaN
144,55.606,13.024,84.17,10.10,NaN,NaN,100608.81,100708.34


In [77]:
sens2 = sens2.astype('float')

In [78]:
sens2.dtypes

latitude                float64
longitude               float64
humidity                float64
temperature             float64
P1                      float64
P2                      float64
pressure                float64
pressure_at_sealevel    float64
dtype: object

In [79]:
humid_values = sens2[sens2['humidity'].notnull()]

In [80]:
locations = humid_values[['latitude', 'longitude']]
weights = humid_values['humidity']

In [81]:
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
fig



Figure(layout=FigureLayout(height='420px'))

In [82]:
value_dict = humid_values.to_dict('index')

In [83]:
value_dict.values()

dict_values([{'latitude': 56.196, 'longitude': 15.094, 'humidity': 99.9, 'temperature': 6.1, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 56.07, 'longitude': 12.698, 'humidity': 99.9, 'temperature': 8.7, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 55.82, 'longitude': 13.03, 'humidity': 99.9, 'temperature': 7.6, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 55.722, 'longitude': 13.202, 'humidity': 99.9, 'temperature': 9.0, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 56.098, 'longitude': 12.65, 'humidity': 100.0, 'temperature': 7.55, 'P1': nan, 'P2': nan, 'pressure': 107191.5, 'pressure_at_sealevel': 107668.46}, {'latitude': 55.606, 'longitude': 13.024, 'humidity': 84.13, 'temperature': 10.1, 'P1': nan, 'P2': nan, 'pressure': 100606.13, 'pressure_at_sealevel': 100705.65}, {'latitude': 55.648, 'longitude': 13.208, 'humidity': 99.9, 'temperatu

In [87]:
data = humid_values.to_dict('index').values()
#data

In [88]:

plant_locations = [(plant['latitude'], plant['longitude']) for plant in data] 
info_box_template = """
<dl>
<dt>Fuktighet</dt><dd>{humidity}</dd>
<dt>Lufttryck</dt><dd>{pressure}</dd>
</dl>
"""

plant_info = [info_box_template.format(**plant) for plant in data]
marker_layer = gmaps.marker_layer(plant_locations, info_box_content=plant_info)

In [89]:
fig = gmaps.figure()

fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
sensors[0]

{'sampling_rate': None,
 'location': {'country': 'SE',
  'altitude': '8.2',
  'id': 8180,
  'indoor': 0,
  'exact_location': 0,
  'longitude': '13.024',
  'latitude': '55.606'},
 'sensordatavalues': [{'value_type': 'temperature',
   'id': 12788794817,
   'value': '7.86'},
  {'value_type': 'pressure', 'id': 12788794818, 'value': '101336.69'},
  {'value_type': 'humidity', 'id': 12788794820, 'value': '78.45'},
  {'value_type': 'pressure_at_sealevel', 'value': 101437.74}],
 'id': 6019977378,
 'timestamp': '2020-01-09 10:48:53',
 'sensor': {'sensor_type': {'name': 'BME280',
   'id': 17,
   'manufacturer': 'Bosch'},
  'id': 16150,
  'pin': '11'}}

Det finns olika typer av värden:

* P1 är PM10, alltså partiklar mindre än 10 mikrometer
* P2 är PM2.5, alltså partiklar mindre än 2,5 mikrometer
* temperature är temperatur
* humidity är luftfuktighet
* pressure är lufttryck